In [1]:
import json
import os

In [2]:
data = []
with open('prepared-wikipedia-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l)

In [3]:
suppose = {'question', 'A', 'B', 'C', 'D', 'answer'}

In [16]:
from glob import glob

files = glob('generate-wikipedia-qa-llama3/*.json')
len(files)

91635

In [17]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [18]:
selected = []
for d_ in d[0].split('```'):
    try:
        selected.append(json.loads(d_))
    except:
        pass

In [19]:
selected

[[{'question': 'Di manakah beribu pejabat Stora Enso Oyj?',
   'A': 'Stockholm, Sweden',
   'B': 'Helsinki, Finland',
   'C': 'Oslo, Norway',
   'D': 'Copenhagen, Denmark',
   'answer': 'B'},
  {'question': 'Apabila syarikat Stora Enso dibentuk?',
   'A': '1980',
   'B': '1998',
   'C': '2001',
   'D': '2010',
   'answer': 'B'},
  {'question': 'Berapa ramai pekerja Stora Enso pada tahun 2018?',
   'A': '20,000',
   'B': '26,000',
   'C': '30,000',
   'D': '35,000',
   'answer': 'B'},
  {'question': 'Di mana terdapat operasi penting Stora Enso?',
   'A': 'Eropah sahaja',
   'B': 'Asia, Amerika Selatan dan Amerika Syarikat',
   'C': 'Afrika dan Australia',
   'D': 'Semua di atas',
   'answer': 'B'},
  {'question': 'Pada tahun 2015, Stora Enso menduduki tempat keberapa dalam jualan?',
   'A': 'Kelima',
   'B': 'Ketujuh',
   'C': 'Kesembilan',
   'D': 'Kesebelas',
   'answer': 'B'},
  {'question': 'Apakah tahun penerbitan sijil saham terpelihara yang tertua di dunia?',
   'A': '1200',
   '

In [22]:
from tqdm import tqdm

with open('generated-wikipedia-qa-llama3.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'generate-wikipedia-qa-llama3/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                selected = []
                for d__ in d_.split('```'):
                    try:
                        selected.append(json.loads(d__))
                    except:
                        pass
                if not len(selected):
                    continue
                qa = selected[0]
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('A' in q and isinstance(q['A'], str)):
                            continue
                        if not ('B' in q and isinstance(q['B'], str)):
                            continue
                        if not ('C' in q and isinstance(q['C'], str)):
                            continue
                        if not ('D' in q and isinstance(q['D'], str)):
                            continue
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
            
        if not len(questions):
            continue
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

100%|████████████████████████████████████████████████████████████████████████| 97963/97963 [00:06<00:00, 14866.83it/s]


In [23]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='generated-wikipedia-qa-llama3.jsonl',
    path_in_repo='generated-wikipedia-qa-llama3.jsonl',
    repo_id='mesolitica/llama-3-70b-qa',
    repo_type='dataset',
)

generated-wikipedia-qa-llama3.jsonl:   0%|          | 0.00/339M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/llama-3-70b-qa/commit/286a50d6181c49dfd3d2ee62908d154a4180bf12', commit_message='Upload generated-wikipedia-qa-llama3.jsonl with huggingface_hub', commit_description='', oid='286a50d6181c49dfd3d2ee62908d154a4180bf12', pr_url=None, pr_revision=None, pr_num=None)